# Global Average
* Outputs the mean score across all (user, item) pairs
  * Prediction for user $i$ and item $j$ is $\tilde r_{ij} = a$
  * Loss function is $L = \sum_\Omega(r_{ij} - \tilde r)^2$
  * $\Omega$ is the set of oberved pairs $(i, j)$
  * $r_{ij}$ is the rating for user $i$ and item $j$
* This is a useful baseline to measure improvements from

In [1]:
name = "GlobalAverage";
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [ ]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

## Training

In [3]:
μ = mean(training.rating);

## Inference

In [4]:
function make_prediction(users, items, μ)
    fill(μ, length(users))
end;

model(users, items) = make_prediction(users, items, μ);

In [5]:
write_predictions(model);

[ Info: 20211204 20:08:46 training set: RMSE 1.7805202512279754 MAE 1.3906332220653705 R2 0.0
[ Info: 20211204 20:08:49 validation set: RMSE 1.7806075310542464 MAE 1.3908660830061252 R2 -2.1652728277032907e-7


In [6]:
write_params(Dict("μ" => μ));